<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap07/7_2_Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 7.2: Backpropagation**

This notebook runs the backpropagation algorithm on a deep neural network as described in section 7.4 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.

In [336]:
import numpy as np
import matplotlib.pyplot as plt

First let's define a neural network.  We'll just choose the weights and biases randomly for now

In [337]:
# Set seed so we always get the same random numbers
np.random.seed(0)

# Number of layers
K = 5
# Number of neurons per layer
D = 6
# Input layer
D_i = 1
# Output layer
D_o = 1

# Make empty lists
all_weights = [None] * (K+1)
all_biases = [None] * (K+1)

# Create input and output layers
all_weights[0] = np.random.normal(size=(D, D_i))
all_weights[-1] = np.random.normal(size=(D_o, D))
all_biases[0] = np.random.normal(size =(D,1))
all_biases[-1]= np.random.normal(size =(D_o,1))

# Create intermediate layers
for layer in range(1,K):
  all_weights[layer] = np.random.normal(size=(D,D))
  all_biases[layer] = np.random.normal(size=(D,1))

In [338]:
# Define the Rectified Linear Unit (ReLU) function
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation

Now let's run our random network.  The weight matrices $\boldsymbol\Omega_{1\ldots K}$ are the entries of the list "all_weights" and the biases $\boldsymbol\beta_{1\ldots k}$ are the entries of the list "all_biases"

We know that we will need the activations $\mathbf{f}_{0\ldots K}$ and the activations $\mathbf{h}_{1\ldots K}$ for the forward pass of backpropagation, so we'll store and return these as well.


In [339]:
def compute_network_output(net_input, all_weights, all_biases):

  # Retrieve number of layers
  K = len(all_weights) -1

  # We'll store the pre-activations at each layer in a list "all_f"
  # and the activations in a second list[all_h].
  all_f = [None] * (K+1)
  all_h = [None] * (K+1)

  #For convenience, we'll set
  # all_h[0] to be the input, and all_f[K] will be the output
  all_h[0] = net_input

  # Run through the layers, calculating all_f[0...K-1] and all_h[1...K]
  for layer in range(K):
      # Update preactivations and activations at this layer according to eqn 7.16
      # Remmember to use np.matmul for matrrix multiplications
      # TODO -- Replace the lines below
      all_f[layer] = all_biases[layer] + np.matmul(all_weights[layer], all_h[layer])
      all_h[layer+1] = ReLU(all_f[layer])

  # Compute the output from the last hidden layer
  # TO DO -- Replace the line below
  all_f[K] = all_biases[K] + np.matmul(all_weights[K], all_h[K])

  # Retrieve the output
  net_output = all_f[K]

  return net_output, all_f, all_h

In [340]:
# Define in input
net_input = np.ones((D_i,1)) * 1.2
# Compute network output
net_output, all_f, all_h = compute_network_output(net_input,all_weights, all_biases)
print("True output = %3.3f, Your answer = %3.3f"%(1.907, net_output[0,0]))


True output = 1.907, Your answer = 1.907


Now let's define a loss function.  We'll just use the least squares loss function. We'll also write a function to compute dloss_doutput

In [341]:
def least_squares_loss(net_output, y):
  return np.sum((net_output-y) * (net_output-y))

def d_loss_d_output(net_output, y):
    return 2*(net_output -y);

In [342]:
y = np.ones((D_o,1)) * 20.0
loss = least_squares_loss(net_output, y)
print("y = %3.3f Loss = %3.3f"%(y, loss))

y = 20.000 Loss = 327.371


/var/folders/b2/xg8329j50lggt263jywktrk80000gn/T/ipykernel_16039/3361061389.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("y = %3.3f Loss = %3.3f"%(y, loss))


Now let's compute the derivatives of the network.  We already computed the forward pass.  Let's compute the backward pass.

In [372]:
# We'll need the indicator function
def indicator_function(x):
  x_in = np.array(x)
  x_in[x_in>=0] = 1
  x_in[x_in<0] = 0
  return x_in

# Main backward pass routine
def backward_pass(all_weights, all_biases, all_f, all_h, y):
  # We'll store the derivatives dl_dweights and dl_dbiases in lists as well
  all_dl_dweights = [None] * (K+1)
  all_dl_dbiases = [None] * (K+1)
  # And we'll store the derivatives of the loss with respect to the activation and preactivations in lists
  all_dl_df = [None] * (K+1)
  all_dl_dh = [None] * (K+1)
  # Again for convenience we'll stick with the convention that all_h[0] is the net input and all_f[k] in the net output

  # Compute derivatives of the loss with respect to the network output
  all_dl_df[K] = np.array(d_loss_d_output(all_f[K],y))

  # Now work backwards through the network
  for layer in range(K,-1,-1):
    # TODO Calculate the derivatives of the loss with respect to the biases at layer this from all_dl_df[layer]. (eq 7.21)
    # NOTE!  To take a copy of matrix X, use Z=np.array(X)
    # REPLACE THIS LINE
    all_dl_dbiases[layer] = np.array(all_dl_df[layer]) # np.zeros_like(all_biases[layer])

    # TODO Calculate the derivatives of the loss with respect to the weights at layer from all_dl_df[layer] and all_h[layer] (eq 7.22)
    # Don't forget to use np.matmul
    # REPLACE THIS LINE
    print(all_dl_df[layer], all_h[layer].transpose())
    all_dl_dweights[layer] = np.matmul(all_dl_df[layer], all_h[layer].transpose())

    # TODO: calculate the derivatives of the loss with respect to the activations from weight and derivatives of next preactivations (second part of last line of eq 7.24)
    # REPLACE THIS LINE
    all_dl_dh[layer] = np.matmul(all_weights[layer].transpose(), all_dl_df[layer]) #np.zeros_like(all_h[layer])


    if layer > 0:
      # TODO Calculate the derivatives of the loss with respect to the pre-activation f (use deriv of ReLu function, first part of last line of eq. 7.24)
      # REPLACE THIS LINE
      all_dl_df[layer-1] = np.array(d_loss_d_output(all_f[layer-1],y))

  return all_dl_dweights, all_dl_dbiases

In [373]:

all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)
print(all_dl_dweights, all_dl_dbiases)

[[-36.187]] [[ 0.     0.     0.     0.    11.063  0.   ]]
[[-54.446]
 [-81.413]
 [-68.655]
 [-69.634]
 [-17.874]
 [-81.605]] [[ 0.    15.662  9.427  0.     4.222  1.946]]
[[-43.761]
 [ -8.677]
 [-21.147]
 [-42.305]
 [-31.556]
 [-36.107]] [[0.    0.    0.765 0.    8.939 2.291]]
[[-48.436]
 [-49.274]
 [-38.469]
 [-59.881]
 [-22.123]
 [-35.417]] [[0.    5.727 0.    0.    3.353 0.   ]]
[[-45.335]
 [-28.546]
 [-45.634]
 [-52.74 ]
 [-33.294]
 [-54.303]] [[2.878 0.602 1.618 3.023 3.735 0.   ]]
[[-34.244]
 [-38.796]
 [-36.763]
 [-33.955]
 [-32.53 ]
 [-42.756]] [[1.2]]
[array([[-41.093],
       [-46.556],
       [-44.116],
       [-40.745],
       [-39.036],
       [-51.307]]), array([[-130.47 ,  -27.286,  -73.368, -137.037, -169.333,    0.   ],
       [ -82.152,  -17.181,  -46.197,  -86.287, -106.623,    0.   ],
       [-131.329,  -27.465,  -73.851, -137.939, -170.448,    0.   ],
       [-151.779,  -31.742,  -85.351, -159.419, -196.99 ,    0.   ],
       [ -95.818,  -20.039,  -53.882, -100.64 

In [369]:
np.set_printoptions(precision=3)
# Make space for derivatives computed by finite differences
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# Let's test if we have the derivatives right using finite differences
delta_fd = 0.000001

# Test the dervatives of the bias vectors
for layer in range(K):
  dl_dbias  = np.zeros_like(all_dl_dbiases[layer])
  # For every element in the bias
  for row in range(all_biases[layer].shape[0]):
    # Take copy of biases  We'll change one element each time
    all_biases_copy = [np.array(x) for x in all_biases]
    all_biases_copy[layer][row] += delta_fd
    network_output_1, *_ = compute_network_output(net_input, all_weights, all_biases_copy)
    network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
    dl_dbias[row] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y)) / delta_fd
  all_dl_dbiases_fd[layer] = np.array(dl_dbias)
  print("-----------------------------------------------")
  print("Bias %d, derivatives from backprop:"%(layer))
  print(all_dl_dbiases[layer])
  print("Bias %d, derivatives from finite differences"%(layer))
  print(all_dl_dbiases_fd[layer])
  if np.allclose(all_dl_dbiases_fd[layer],all_dl_dbiases[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")



# Test the derivatives of the weights matrices
for layer in range(K):
  dl_dweight  = np.zeros_like(all_dl_dweights[layer])
  # For every element in the bias
  for row in range(all_weights[layer].shape[0]):
    for col in range(all_weights[layer].shape[1]):
      # Take copy of biases  We'll change one element each time
      all_weights_copy = [np.array(x) for x in all_weights]
      all_weights_copy[layer][row][col] += delta_fd
      network_output_1, *_ = compute_network_output(net_input, all_weights_copy, all_biases)
      network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
      dl_dweight[row][col] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dweights_fd[layer] = np.array(dl_dweight)
  print("-----------------------------------------------")
  print("Weight %d, derivatives from backprop:"%(layer))
  print(all_dl_dweights[layer])
  print("Weight %d, derivatives from finite differences"%(layer))
  print(all_dl_dweights_fd[layer])
  if np.allclose(all_dl_dweights_fd[layer],all_dl_dweights[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")

-----------------------------------------------
Bias 0, derivatives from backprop:
[[-34.244]
 [-38.796]
 [-36.763]
 [-33.955]
 [-32.53 ]
 [-42.756]]
Bias 0, derivatives from finite differences
[[ -4.486]
 [  4.947]
 [  6.812]
 [ -3.883]
 [-24.935]
 [  0.   ]]
Failure!  Derivatives different.
-----------------------------------------------
Bias 1, derivatives from backprop:
[[-45.335]
 [-28.546]
 [-45.634]
 [-52.74 ]
 [-33.294]
 [-54.303]]
Bias 1, derivatives from finite differences
[[  0.   ]
 [-11.297]
 [  0.   ]
 [  0.   ]
 [-10.722]
 [  0.   ]]
Failure!  Derivatives different.
-----------------------------------------------
Bias 2, derivatives from backprop:
[[-48.436]
 [-49.274]
 [-38.469]
 [-59.881]
 [-22.123]
 [-35.417]]
Bias 2, derivatives from finite differences
[[ 0.   ]
 [ 0.   ]
 [ 0.938]
 [ 0.   ]
 [-9.993]
 [ 0.508]]
Failure!  Derivatives different.
-----------------------------------------------
Bias 3, derivatives from backprop:
[[-43.761]
 [ -8.677]
 [-21.147]
 [-42.30